<a href="https://colab.research.google.com/github/morenourey/ENTREGABLES/blob/main/transf_learning_catsanddogs_basic_v0_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# Connectar a Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Librerias

In [ ]:
from tensorflow import keras as ks
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os

from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Model

# Variables de entorno

<font color="#FF0000">Aquí definireis vuestra ruta del proyecto de perros y Gatos</font>

In [ ]:
ANCHO_IMAGEN = 150
ALTURA_IMAGEN=150
IMAGE_SIZE = (ANCHO_IMAGEN, ALTURA_IMAGEN)
CANALES_IMAGENES = 3

ruta_archivos = '/content/drive/MyDrive/00_Nuclio_DS_Master/Dogs-vs-cats/'

# Cargaremos datos en NPZ


In [ ]:
dict_npz = np.load(ruta_archivos+'xy_train_img.npz')
x_train_img = dict_npz['x']
y_train_img = dict_npz['y']

dict_npz = np.load(ruta_archivos+'xy_test_img.npz')
x_test_img = dict_npz['x']
y_test_img = dict_npz['y']

dict_npz = np.load(ruta_archivos+'xy_val_img.npz')
x_val_img = dict_npz['x']
y_val_img = dict_npz['y']

In [ ]:
x_train_scaled = x_train_img / 255.
x_test_scaled = x_test_img / 255.
x_val_scaled = x_val_img / 255.

In [ ]:
y_train_img

# Montamos la Red Neuronal

## Cargar la parte de extraccion de features de VGG16

In [ ]:
# Creemos una red que será extracción de features basada en VGG16 entrenada con ImageNet

In [ ]:
# Veamos como va lo de "congelar" capas de entrenamiento

## Pre-procesaremos las imagenes originales, pasandolas por VGG16, para poder usarlas en la parte de clasificacion

In [ ]:
# Llevamos las imagenes desde su origen al cuello de botella de VGG16, justo después del Flatten()

# Montemos la red de clasificación

In [ ]:
# Definamos el output de salida de la transformacion VGG16
output_from_vgg16 = 0 #TODO

In [ ]:
model_post_vgg = ks.Sequential()

model_post_vgg.add(ks.layers.InputLayer(input_shape=(output_from_vgg16)))
model_post_vgg.add(ks.layers.Dense(512, activation='relu', input_shape=(output_from_vgg16,)))
model_post_vgg.add(ks.layers.Dropout(0.3))
model_post_vgg.add(ks.layers.Dense(512, activation='relu'))
model_post_vgg.add(ks.layers.Dropout(0.3))
model_post_vgg.add(ks.layers.Dense(1, activation='sigmoid'))

model_post_vgg.summary()

# Creamos un optimizador

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
new_adam = Adam(learning_rate=0.01)


In [ ]:
model_post_vgg.compile(loss='binary_crossentropy', optimizer=new_adam, metrics=['accuracy'])

# Early stopping!!

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=5)

# Encoding de etiquetas

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(y_train_img)
y_train_encoded = le.transform(y_train_img)
y_val_encoded = le.transform(y_val_img)
y_test_encoded = le.transform(y_test_img)

In [ ]:
print(np.array(y_train_img))
print(np.array(y_train_encoded))

y_train_encoded

# Entrenar

In [ ]:
history = model_post_vgg.fit(x=x_train_features_vgg, y=y_train_encoded, batch_size=512,
                             epochs=10, callbacks=[callback],
                             validation_data=(x_val_features_vgg, y_val_encoded))

# Guardamos el modelo

Aquí tenemos que guardar el modelo simple, pero tambien hemos de guardar, o bien el modelo de preproceso o bien los datos de test procesados en NPZ...

In [ ]:
# Modelo post VGG16
model_post_vgg.save(ruta_archivos+'model_with_basic_transfer_learning_post_vgg16.h5')

# Datos procesados de test
np.savez_compressed(ruta_archivos+'post_vgg16_test_data.npz', x=x_test_features_vgg, y=y_test_encoded)


# Miremos que tal rinde el modelo en errores y accuracy

In [ ]:
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
plt.show()

plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='orange', label='test')
plt.show()

In [ ]:
_, acc = model_post_vgg.evaluate(x_test_features_vgg, y_test_encoded, verbose=0)
print('Modelo con Basic Transfer Learning > %.3f' % (acc * 100.0))
